In [1]:
import pymysql

connection = pymysql.connect(host='localhost',
                             user='Luis',
                             password='Kalibu819@',
                             database='DB_Course')
# it's good to select now the database for you don't need to select which
# database and which table you are querying

cursor = connection.cursor()